# Eezzy Regression
### Analyze a dataset of house prices with Eezzy functionality
Eezzy ties in with powerful libraries like `pandas` and `scikit-learn` in order to allow data scientists to rapidly prototype their machine learning workflow. Within this notebook we'll perform exploratory data analysis on sold houses from 2014 to 2015, located in King County, USA.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import re
import requests
import io
sns.set_style('white')

## Data Preview
Before getting our hands dirty with actual machine learning, we should first develop a understanding of the data at hand. A couple of things must be addressed at first:
- Is there missing data?
- Do some features have high collinearity?
- What format is the data in?

In [2]:
url_name = "https://raw.githubusercontent.com/3Blades/notebook-templates/master/python/Datasets/Housing_Prices.csv"
request=requests.get(url_name).content
df= pd.read_csv(io.StringIO(request.decode('utf-8')))
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [ ]:
df.info()

_Looks like most of the data is immediately usable (numbers), although the date is in a string format. There are no NaN values_

In [ ]:
df.describe()

_The average house in King County is $540,000, with around 3 bedrooms, 2 bathrooms, and 2080 sq ft._

In [ ]:
def assess_collinearity(df):
    combinations = []
    collinearity = df.corr()
    columns = collinearity.columns
    for row in collinearity.itertuples(name=None):
        index = row[0]
        corr = row[1:]
        for idx, value in enumerate(corr):
            if value > .75 and index != columns[idx] and \
                    [columns[idx], index, value] not in combinations:
                combinations.append([
                    index,
                    columns[idx],
                    value
                ])
    print('Potentially Unsafe Features (High Collinearity):')
    for risk in combinations:
        print("The features {0} and {1} have a correlation rating of {2}".format(*risk))

    fig, ax = plt.subplots(figsize=(15, 15))
    corr_arr = collinearity.values.ravel()
    vmin = corr_arr.min()
    vmax = corr_arr.max()
    cax = ax.matshow(collinearity, vmin=vmin, vmax=vmax, cmap='BuGn')
    fig.colorbar(cax)
    ticks = np.arange(0, len(collinearity), 1)
    plt.xticks(ticks, columns, rotation=90, size='large')
    plt.yticks(ticks, columns, size='large')
    ax.grid(False)
    plt.show()
assess_collinearity(df)

### Initial Impressions

Looks like we'll need to watch out for high correlations between `sqft_living` and `sqft_above`. Additionally, we learned that our data is in an excellent format for the most part, although the feature `yr_renovated` is almost entirely empty, and could probably be best combined with another.

---

## Visualize the patterns
Now that we have a sense of what the dataset is like, let's visualize how the house price is dependent on various features, and what trends we may see. Eezzy provides functionality for easy plotting.

In [ ]:
from tbs.plotting.tbs_plotly_plot import plot_time_series
plot_time_series(df[:1000], 'date', 'price', False)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

weekly_mean_df = df.resample('W', on='date').mean()
weeks, weekly_means = weekly_mean_df.index, weekly_mean_df['price']
plt.plot(weeks, weekly_means)

In [ ]:
df[df['date'] > '2015/05/15']['price']

So even though the mean price appears to take off towards the end of the timespan of the dataset, it's only because of an outlier at the very end, and the fact that there are only two houses sold in the final week of the data. So the time of the year a house is sold doesn't appear to have a direct impact on the price the house was sold for.

In [ ]:
import tbs.plotting.tbs_plot as viz
continuous_columns = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view', 'condition']
reload(viz)
viz.continuous_plots(df, continuous_columns, num_columns=3, figsize=(15, 12))

In [ ]:
## create a heatmap with longitude and latitude
def heat(df, feature_names):
    X, Y, V = tuple(df[name].values for name in feature_names)

    xmin = X.min()
    xmax = X.max()
    ymin = Y.min()
    ymax = Y.max()
    vmin = V.min()
    vmax = V.max()

    plt.figure(figsize=(10, 5))
    plt.subplots_adjust(hspace=0.5)
    plt.hexbin(X, Y, C=V, vmin=vmin, vmax=vmax, cmap=plt.cm.YlOrRd_r)

    plt.colorbar()
    plt.axis([xmin, xmax, ymin, ymax])
    plt.title('{2} over {0} and {1}'.format(*feature_names), size='xx-large')
    plt.xlabel(feature_names[0], size='x-large')
    plt.ylabel(feature_names[1], size='x-large')

    plt.show()
heat(df, ('long', 'lat', 'price'))

### Plotting Averages
`plot_mean_over_x` will display the mean y value (in this case price) over x, which can also be a categorical feature.

In [ ]:
def _averages(X, y, slices):
    x_axis, dist, averages, stderrs = tuple(np.ones(slices.shape) for _ in range(4))
    for i, left_index in enumerate(slices):
        left_index = slices[i]
        if i + 1 == len(slices):
            right_index = -1
        else:
            right_index = slices[i + 1]
        x_axis[i] = X[left_index:right_index].mean()
        dist[i] = X[left_index: right_index].shape[0]
        averages[i] = y[left_index:right_index].mean()
        stderrs[i] = y[left_index:right_index].std()
    return x_axis, dist, averages, stderrs

def plot_running_mean(df, x_name, y_name, groups=20):
    from matplotlib.patches import Patch
    
    g = sns.JointGrid(x=x_name, y=y_name, data=df)
    X, y = df[x_name].values, df[y_name].values
    
    uniques = np.unique(X)
    indices = np.argsort(X)
    X = X[indices]
    y = y[indices]
    
    step = len(X) // groups    
    slices = np.arange(0, len(X), step)
    x_axis, dist, averages, stderrs = _averages(X, y, slices) 
    
    def joint(*args):
        plt.plot(x_axis, averages)
        
    def marginal(*args, **kwargs):
        plt.plot(x_axis, stderrs, color='#d16e6e')

    joint_legend = Patch(color='C0', label=y_name)
    ax_legend = Patch(color='#d16e6e', label='stderr')
    
    g.plot_joint(joint)
    g.plot_marginals(marginal)
    g.ax_marg_y.clear()
    g.ax_marg_y.set_axis_off()

    g.ax_joint.set_ylim(min(averages), max(averages))
    plt.legend(handles=[joint_legend, ax_legend])
    plt.suptitle('{0} over {1}'.format(x_name, y_name))
    plt.show()

In [ ]:
def plot_category_mean(df, x_name, y_name, error=True):
    X, y = df[x_name].values, df[y_name].values
    uniques = set(X)
    averages = [y[X == unique].mean() for unique in uniques]
    stderrs = [y[X == unique].std() for unique in uniques]
    x_axis = np.arange(len(uniques))
    if error:
        plt.bar(x_axis, averages, yerr=stderrs)
    else:
        plt.bar(x_axis, averages)
    plt.xticks(x_axis, uniques, multialignment='center')
    plt.xlabel(x_name)
    plt.ylabel(y_name)
    plt.suptitle('{0} over {1}'.format(x_name, y_name))
    plt.show()

In [ ]:
plot_running_mean(df, 'yr_built', 'price')
plot_category_mean(df, 'condition', 'price')
plot_category_mean(df, 'grade', 'price')
plot_category_mean(df, 'view', 'price')
plot_running_mean(df, 'sqft_living', 'price')

_Looks like the 1940s was a rough time to have a house_

_Grade has an excellent fit to average price, as does square feet of living_

## Slimming Down the Number of Features
### The Curse of Dimensionality
Our dataset has several features that are rather redundant or display no correlation with price. In order to improve the accuracy of the model, we'll remove or alter these features that could otherwise distract the model and cause it to underperform.

In [ ]:
# First off, we don't need separate features for year built and year renovated
# especially considering most houses haven't been renovated, meaning that
# the renovated year feature is largely useless. So instead, we'll combine the two

years_built = df['yr_built'].values
years_ren = df['yr_renovated'].values
def update_years(built, ren):
    return built if built > ren else ren
update_years = np.vectorize(update_years)

new_feature = update_years(years_built, years_ren)


In [ ]:
# Next we'll remove data points that aren't relevant for predicting the price
train_df = df.drop([
        'id',
        'date',
        'yr_renovated',
        'view', # no good explanation for what this feature actually is
        'condition', # grade offers a better insight than this does
        'zipcode', # we already have longitude, latitude, and near-15 for geographic data
        'sqft_above'
    ], axis=1)
# and finally, add in the newly composed feature
train_df['yr_built'] = new_feature
train_df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
names = train_df.columns
scaler = StandardScaler()
scaled_train_df = train_df.copy()
for name in names:
    if name != 'price':
        scaled_train_df[name] = scaler.fit_transform(scaled_train_df[name].values.reshape(-1, 1))
scaled_train_df.head()

In [ ]:
train_df.info()

_That's a lot better - now the data is completely prepped and ready to be trained upon_

## Quickly Identifying Well-Performing Models
Eezzy has the convinent method `eezzy_ml` that will take your dataset, assume what sort of problem it is (in this case regression), and score various models with various metrics.

In [ ]:
from tbs.machine_learning import eezzy_ml
exclusions = [
    'AdaBoostRegressor',
    'BaggingRegressor',
    'ExtraTreesRegressor',
    'GradientBoostingRegressor',
    'RandomForestRegressor',
    'GaussianProcessRegressor',
    'KernelRidge',
    'HuberRegressor',
    'SGDRegressor',
    'RadiusNeighborsRegressor',
    'LinearSVR'
]
eezzy_ml(scaled_train_df, y='price', excluded=exclusions, reduce=200)

## Selecting a Model and Refining Results
---
Looks like decision trees and k-nearest neighbors fit the data the best. We'll import the model `DecisionTreeRegressor`, as it performed considerably better than others in the brief spotchecking and it provides an easy way of checking out feature importances. 

In order to get optimal results from the Decision Tree model, we'll try to create interaction variables in order to pull out any additional meaning from the data. We'll train it on all of the possible interaction variable features, and select the "most important" ones out of that

In [ ]:
from tbs.feature_engineering.tbs_transform import create_interactions
interactions_df = create_interactions(train_df.drop(['price', 'bathrooms'], axis=1))

for name in interactions_df.columns:
    if np.any(np.isinf(interactions_df[name])):
        interactions_df = interactions_df.drop(name, axis=1)

for name in interactions_df.columns:
    result = scaler.fit_transform(interactions_df[name].values.reshape(-1, 1))
    interactions_df[name] = result
interactions_df.head()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_predict
from tbs.plotting.classification_plot import feature_importances

best_params = {'criterion': 'mae', 'min_impurity_split': 1e-07}
tree = DecisionTreeRegressor(**best_params)
tree.fit(interactions_df.values, df['price'].values)
feature_importances(interactions_df, tree)

Now we'll pull out the more important features and then compare the `scaled_train_df`, which has our normal data, and `interactions_df`, which has the interaction variables.

In [ ]:
importances = tree.feature_importances_
indices = np.argsort(importances)[::-1]
indices = indices[importances > 0.01]
names = interactions_df.columns[indices]
interactions_df[names].head()

### Decision Tree

In [ ]:
from sklearn.metrics import r2_score
pred_y = cross_val_predict(tree, interactions_df.values, train_df['price'].values)
r2_score(train_df['price'].values, pred_y)

In [ ]:
pred_y = cross_val_predict(tree, scaled_train_df.drop(['price'], axis=1).values, train_df['price'].values)
r2_score(train_df['price'].values, pred_y)

### K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
best_params = {'p': 1, 'leaf_size': 15, 'algorithm': 'ball_tree', 'n_neighbors': 5}
neighbors = KNeighborsRegressor(**best_params)

pred_y = cross_val_predict(neighbors, interactions_df.values, train_df['price'].values)
r2_score(train_df['price'].values, pred_y)

In [ ]:
pred_y = cross_val_predict(neighbors, scaled_train_df.drop(['price'], axis=1).values, train_df['price'].values)
r2_score(train_df['price'].values, pred_y)